<h1 style=\"text-align: center; font-size: 50px;\"> 📜 Text Generation with Neural Networks and Tensorflow</h1>

In this notebook our objective is to demonstrate how to generate text using a character-based RNN and Tensorflow working with a dataset of Shakespeare's  writing

# Notebook Overview
- Imports
- Configurations
- Generating Predictions

## Imports

In [1]:
# ------------------------ Standard Library Imports ------------------------
import logging
import warnings
from pathlib import Path
from datetime import datetime

# ------------------------ Third-Party Libraries ------------------------
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GRU, InputLayer
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
import torch
from torch import nn


2025-06-13 00:14:56.722889: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-13 00:14:56.731996: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749773696.741781     721 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749773696.744651     721 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 00:14:56.755964: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Configurations

In [2]:
warnings.filterwarnings("ignore")

In [3]:
# === Create logger ===
logger = logging.getLogger("text-generation-TF-notebook")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                             datefmt="%Y-%m-%d %H:%M:%S") 

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [4]:
# Define global experiment and run names to be used throughout the notebook
MODEL_NAME = "tf_rnn_model.h5"

# Set up the paths
DATA_PATH = "../data/shakespeare.txt"
TENSORBOARD_PATH = "/phoenix/tensorboard/tensorlogs"


# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [5]:
logger.info('Notebook execution started.')

2025-06-13 00:14:59 - INFO - Notebook execution started.


In [6]:
start_time_all_execution = datetime.now() # This variable is to help us to see in how much time this notebook will run

### Verify Assets

In [7]:
# Check whether the Dataset file exists
is_dataset_available = Path(DATA_PATH).exists()

# Log the configuration status of the dataset
if is_dataset_available:
    logger.info("The Dataset is properly configured.")
else:
    logger.info(
        "The Dataset is not properly configured. Please check if Dataset was downloaded"
        "in your project on AI Studio."
    )

2025-06-13 00:14:59 - INFO - The Dataset is properly configured.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Get Text Data

This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [9]:
path_to_file = DATA_PATH
text = open(path_to_file, 'r').read()

In [10]:
logger.info('First 600 chars: \n')
print(text[:600])

2025-06-13 00:14:59 - INFO - First 600 chars: 




                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else th


## Preparing textual data

We need to encode our data to give the model a proper numerical representation of our text.

In [11]:
# creates a set of unique characters found in the text
vocab = sorted(set(text))

### Text Vectorization

In [12]:
char_to_int = {u:i for i, u in enumerate(vocab)}
# assigns a unique integer to each character in a dictionary format, 
# creating a mapping that can later be used to transform encoded predictions back into characters

In [13]:
int_to_char = np.array(vocab)
# reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers, which is used to encode the text.

In [14]:
encoded_text = np.array([char_to_int[c] for c in text])
# encodes the entire text as an array of integers, with each integer representing the character at that position
# in the text according to the encoder dictionary

## Creating Training Batches

Training batches are a way of dividing the dataset into smaller, manageable groups of data points that are fed into a machine learning model during the training process.

In [15]:
seq_len = 120 # length of sequence for a training example
total_num_seq = len(text)//(seq_len+1) # total number of training examples

# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

I0000 00:00:1749773703.548590     721 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2268 MB memory:  -> device: 0, name: NVIDIA T600, pci bus id: 0000:01:00.0, compute capability: 7.5


In [16]:
def create_seq_targets(seq):
    """
    Function that takes a sequence as input, duplicates, and shifts it to align the input and label. 

    Args:
        seq: sequence of characters

    Returns:
        The text input and corresponding target.
    """
    try:
        input_txt = seq[:-1]
        target_txt = seq[1:]
        return input_txt, target_txt
    except Exception as e:
            logger.error(f"Error creating sequences of targets: {str(e)}")

In [17]:
dataset = sequences.map(create_seq_targets)

In [18]:
# Batch size
batch_size = 128
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## Creating the GRU Model

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embed_dim = 64
# Number of RNN units
rnn_neurons = 1026

In [20]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [21]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    """Architecture to create the model.

    Args:
        vocab_size: Length of the vocabulary in chars.
        embed_dim: Embedding dimension.
        rnn_neurons: Number of RNN units.
        batch_size: Size of the batchs.

    Returns:
        Model.
    """
    try:
        model = Sequential()
        model.add(InputLayer(batch_shape=(batch_size, None)))
        
        model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim))

        model.add(GRU(rnn_neurons,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))

        model.add(Dense(vocab_size))
        model.compile(optimizer='adam', loss=sparse_cat_loss)
        logger.info("Model architecture created successfully")
        return model
    except Exception as e:
            logger.error(f"Error creating model architecture: {str(e)}")

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size)
model.summary()

2025-06-13 00:15:04 - INFO - Model architecture created successfully


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (128, None, 64)        │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (128, None, 1026)      │     3,361,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (128, None, 84)        │        86,268 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,452,820 (13.17 MB)

 Trainable params: 3,452,820 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

## Instance of the Model

In [22]:
model = create_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    rnn_neurons=rnn_neurons,
    batch_size=batch_size
)


2025-06-13 00:15:04 - INFO - Model architecture created successfully


In [23]:
# TensorBoard
log_dir = TENSORBOARD_PATH
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

## Training the model

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


I0000 00:00:1749773705.301230     817 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-13 00:15:05.499968: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
epochs = 20
model.fit(dataset,epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 112s 316ms/step - loss: 3.0007
Epoch 2/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 103s 290ms/step - loss: 1.6809
Epoch 3/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 225s 640ms/step - loss: 1.4274
Epoch 4/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 221s 626ms/step - loss: 1.3257
Epoch 5/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 228s 646ms/step - loss: 1.2699
Epoch 6/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 222s 630ms/step - loss: 1.2326
Epoch 7/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 237s 673ms/step - loss: 1.2072
Epoch 8/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 241s 684ms/step - loss: 1.1840
Epoch 9/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 237s 673ms/step - loss: 1.1658
Epoch 10/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 513s 1s/step - loss: 1.1479 
Epoch 11/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 105s 296ms/step - loss: 1.1350
Epoch 12/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 103s 290ms/step - loss: 1.1208
Epoch 13/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 221s 630ms/step - loss: 1.1074
Epoch 14/20
351/351 ━━━━━━━━━━━━━━━━━━━━ 225s 639ms/step - loss: 1.0960
Epo

## Saving the Model

In [27]:
model_name = MODEL_NAME

In [28]:
model.save(f'models/{model_name}') 
logger.info("Model saved")

2025-06-13 01:30:31 - INFO - Model saved


## Load Model

In [29]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights(f'models/{model_name}')

model.build(tf.TensorShape([1, None]))

2025-06-13 01:30:31 - INFO - Model architecture created successfully


# Generating Predictions

In [30]:
def generate_text(model, start_seed="The ", gen_size=100, temp=1.0):
    """
    Generates a sequence of text using the trained character-level language model.

    Args:
        model: Model created on function create_model
        start_seed: Set of characters that will be the beginning of the text. 
        gen_size : Number of characters. Defaults to 100.
        temp: Controls the randomness of the predictions made by the model.

    Returns:
        The full generated text including the seed and the newly predicted characters.
    """
    try:
        num_generate = gen_size
        input_eval = [char_to_int[s] for s in start_seed]
        input_eval = tf.expand_dims(input_eval, 0)
        text_generated = []
        temperature = temp


        for i in range(num_generate):
            predictions = model(input_eval)
            predictions = tf.squeeze(predictions, 0)
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
            input_eval = tf.expand_dims([predicted_id], 0)
            text_generated.append(int_to_char[predicted_id])

        return start_seed + ''.join(text_generated)
    except Exception as e:
            logger.error(f"Error making predictions: {str(e)}")


#### Generating a text with 1000 chars starting with word 'Confidence'

In [31]:
for layer in model.layers:
    if hasattr(layer, 'reset_states'):
        layer.reset_states()
print(generate_text(model, start_seed="Confidence ", gen_size=1000))

Confidence with virtues, joyful mortals of peacef will set uhespection full.
                                 Exeunt you at Criention, command, become
    And set thee to thy youngest date; and like these course
    I found it was with loyal scattering rare
    With me to lip    Exit.
  EMILIA. I wonder, sir, but tell; my lord:  
  YORK. So was I have drunk new his victory; but I'll practice
    You breathe together; I will grow to cold
    And put the blessed earl outwear and cull thee,
    But on that perish cannot burn and quite.


                          86A. Wife, well?
  HECTOR. I am in Roderis that loves Cassio.
  PROSPERO. And, Emilia.
  SEBASTIAN. Not God apprehends your thump.
  BASSANIO. Ingin DALCO. I take the lady.
  CRESSIDA. Then must accuse you!
  ANTONIO. Nor I honor without the sad kept o' th' sea
    As friends a dream in amongst the very
    Obedience. Turn to counsel Cassio yourselves.
    But come, sirs, answer me her reporting,
    Although it be dead man?
  AG

#### Generating a text with 1000 chars starting with word 'Love'

In [32]:
print(generate_text(model, start_seed="Love ", gen_size=1000))

Love me one Nile; he was so iglant.
    So high description, sent her funeral.
  LEONTES. Lead up their kind'st tatting tears will I talk of you that thou mightst thou have.                 [Dives from me: then 'tis a gentleman, Lucions

Enter LEONTES, TRINCUS, LAUNCENOT WIDOW. Heaven,
    rancod. Best did lock us the charmiest of thy hand
    The words they are already, not afraid of love.  
    Two If thy hand commands thy love, look,
    No prince!- as, good Craus! Pray you! let me see?
  CURDINAL. Nay, hear me, Peter Henry's loving friends.
  ANTONIO. How now, my lord, is't not that I have done.
  DIANA. Hangworph is at all locks
    in modesty; and endeavour, and yours are edre I lie to-night
    By counting up the way to wife, so fast
    When the rarent dragon that hath seore to bed.
    Yet; dew we thought it persons to you;
    Death in the bunch of heaven would come to ourselves
    And you affract e'er undergrat'st my stheep.  
    That we or a foully drunk be end.
    Go te

In [33]:
logger.info(f' {datetime.now() - start_time_all_execution}')

2025-06-13 01:30:44 - INFO -  1:15:45.479229


In [34]:
logger.info('Notebook execution completed.')

2025-06-13 01:30:44 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.